In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4 

testset = torchvision.datasets.CIFAR10(root='./dataset', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
from PIL import Image
import glob
image_list = {}
for filename in glob.glob('./dataset/animal/*/*.jpg'):
    im=Image.open(filename)
    image_list[filename] = im

In [ ]:

# Step 1: Initialize model with the best available weights
weights = EfficientNet_B3_Weights.DEFAULT
model = efficientnet_b3(weights=weights)
model.eval()

# # Step 2: Initialize the inference transforms
preprocess = weights.transforms()


In [ ]:
correct_pred = {classname: 0 for classname in weights.meta["categories"]}
total_pred = {classname: 0 for classname in weights.meta["categories"]}
# again no gradients needed
with torch.no_grad():
    for i, data in enumerate(testloader):
        if i == 8:
            break
        images, labels = data
        # outputs = net(images)

        # # Step 3: Apply inference preprocessing transforms
        batch = preprocess(images).unsqueeze(0)
        
        # Step 4: Use the model and print the predicted category
        prediction = model(images).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        predicted = weights.meta["categories"][class_id]

        # collect the correct predictions for each class
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
        
        if i % 2000 == 1999:    # print every 2000 iterations
            print(f'[{i + 1:5d}]')


In [ ]:

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
class_id

In [ ]:
 len(weights.meta["categories"])